In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup

In [5]:
b=webdriver.Firefox()
b.get('https://facebook.com')
login = b.find_element_by_id('email')
login.send_keys('email')#typing facebook login
passw = b.find_element_by_id('pass')
passw.send_keys('password')#typing facebook password
k=b.find_element_by_css_selector('input[type="submit"]')
k.click()

# Page parsing

In [6]:
def get_frends(nikname,max_frends=250):
    max_frends = int(max_frends/12)
    #make links to friends NikName
    link_to_friends = 'https://facebook.com/'+nikname+'/friends'
    #get content friends
    b.get(link_to_friends)
    #scroll down becouse download all content
    html = b.find_element_by_tag_name('html')
    for i in range(max_frends):
        html.send_keys(Keys.END)
        sleep(3)
    #grab class when is information about frends    
    fr = b.find_elements_by_xpath('//div[@class="fsl fwb fcb"]')   
    #prepare information
    friends={}
    #loop folow content
    for i in range(len(fr)):
        fr_link = fr[i].find_element_by_tag_name('a')
        link = fr_link.get_attribute('href')
        
        if link.rfind('?fref')>0 and link.rfind('hc_location=friends_tab')>0:
            friends[link[len('https://www.facebook.com/'):link.rfind('?fref')]] = str(fr[i].text)
        elif link.rfind('&fref') and link.rfind('hc_location=friends_tab')>0:   
            friends[link[link.rfind('&fref')-15:link.rfind('&fref')]] = str(fr[i].text)
     
                
    return friends 

In [15]:
#количество друзей
def get_count_friends(login):
    sleep(3)
    link_friends = link_friends = 'https://www.facebook.com/'+login+'/friends_all' #number of friends
    b.get(link_friends)
    count_fr = b.find_elements_by_xpath('//div[@class="_3dc lfloat _ohe _5brz _5bry"]/a[@id="u_0_1i"]/span[@class="_3d0"]')   
    return int(count_fr[0].text)

# Reading data

In [8]:
import pandas as pd

In [1]:
start_login = 'login'               #first login for start
start_fio = 'First and last name'   #First and last name for start

In [16]:
columns =['NK1','FL_name1','NK2','FL_name2']
df = pd.DataFrame(columns=columns)

frends0 = get_frends(start_login,get_count_friends(start_login))

for f0 in frends0:
    sleep(10) 
    df.loc[len(df)]={'NK1': start_login,'FL_name1': start_fio,'NK2': f0,'FL_name2': frends0[f0]}
    frends1 = get_frends(f0,get_count_friends(f0))
    for f1 in frends1:
        sleep(10)                 
        df.loc[len(df)]={'NK1': f0,'FL_name1': frends0[f0] ,'NK2': f1,'FL_name2': frends1[f1]} 

df.to_csv('graf_fb_'+start_login, index=False)

IndexError: list index out of range